# Week 1 presentation

The first week is pretty much about setting everything up... 

## login for euler cluster and try to set up login without using password 

the first thing is to login ssh jiaychen@euler.ethz.ch account and conduct basic commands in bash 

### Some basic bash commands

In [ ]:
cd # change current directory;
cd .. # to the parent directory of current directory


In [ ]:
pwd # print current working directory

In [ ]:
mkdir #create a new directory

In [ ]:
ls # print a list of file or working 
ls -lyrth
ls -lyath

In [ ]:
rm # remove
rm * #remove every file in the current directory
rm -r # remove a directory

In [ ]:
cp # copy a file 
cp [file name] [directory to be moved to]
cp -r #copy a directory

In [ ]:
touch # create a file 

In [ ]:
vi #using vim to edit file
i #insert
ZZ #exit
[control]V #paste

### Set up euler using ssh keys without password

first thing would be creating ssh key using following commands:


In [ ]:
ssh-keygen -t ed25519 -f $HOME/.ssh/id_ed25519_euler  
ssh-copy-id -i $HOME/.ssh/id_ed25519_euler.pub    jiaychen@euler.ethz.ch
ssh-add --apple-use-keychain ~/.ssh/id_ed25519_euler

then create a config file in the directory of .ssh

In [ ]:
vi config

then edit the config with following commands:

In [ ]:
Host euler.ethz.ch
    HostName euler.ethz.ch
    User avillas

Host * 
   UseKeychain yes 
   AddKeysToAgent yes 
   IdentityFile ~/.ssh/id_ed25519_euler

##Then run this outside of the config
ssh-add -K ~/.ssh/id_ed25519_euler

with everything settled correctly, then euler cluster is avaliable to login without password

## Basic understanding of FASTQ, BAM and VCS

### FASTQ
FASTQ file is a file that split the whole genome sequence into 150 bp fragments
- 1st line is the name of the read: what sequence machine created, where the gene located in the cell
- 2nd line is the 150 bp of the DNA
- 3rd line of the + for aesthetics
- 4th line: quality score of the read (how confident the base call is correct)

one limitation of FASTQ is that we don't know the exact location of the sequences

### BAM
When sequences are aligned using FASTQ file and compare to a reference genome, the whole genome sequence is stored in the BAM file (binary) and therefore may need to change to SAM file to be readable.

example SAM file
![SAM](notes/SAM.png)
For each of the column:
1. Query name: identify any individual reads
2. FLAG value: flag score 
3. Reference name: SQ line
4. Position: 
5. Mapping quality: confident score 
6. CIGAR: continuity or discontinuity in alignment (Match, deletion, insertion)
7. Reference name for mate; “=” if identical to the reference name value
8. Position of mate (analogous to 4)
9. Template length: length of template sequence 
10. Sequence
11.	Quality string
12.	Predefined tags: additional info for the alignment/ read

### VCF

VCF file contains all the difference in base pairs for the individual comparing to the reference genome
for the different column:
![VCF](notes/VCF.png)

- #CHROM: number of chromosomes
- POS: position on the chromosome
- ID: names (rs ids)
- REF:reference base
- ALT: variant base
- QUAL: A quality score 
- FILTER: see the variation failed or PASS a given set of filters
- INFO: key-values describing the variation 
- FORMAT: list of fields describing the data
- SAMPLEs: values given for the FORMAT


## Other settings in cluster


### Set Conda in cluster

download miniconda locally and then using filezilla to move the file to euler.ethz.ch using the same username and password to login euler

To activate conda on the cluster, type:

In [ ]:
conda activate
conda deactivate # the (base) in front of username would be disappear.

bcftools is used in VCF and BCF, to install the package of bcftools, we need to run the commands in the cluster

In [ ]:
conda install -c bioconda bcftools

### Generating jupyter notebooks and synchronizing to Github

create a ETH_jupyter folder in github then install jupyter in the environment using following codes:

In [ ]:
pip3 install jupyter
pip3 install --upgrade jupyter ##If already installed for updating it

#### synchronizing (this part is not ready yet but will keep working on)

In [ ]:
git config --global user.email jadechen212@gmail.com
git config --global user.name jadechen212
##These steps are generally not needed as it was already defined and stays like this
cd /cluster/home/avillas/
mkdir Jupyter_au
cd /cluster/home/avillas/Jupyter_au
git init #this step is necessary if starting the repository from a remote folder
git remote add origin https://github.com/Audald/Jupyter_au #This is the default http source and how I work locally. However, I had to change it to the below one for Leonhard as I wanted the SSH key not to be asked for user and password all the time.
git remote set-url origin git@github.com:Audald/Jupyter_au.git #This needs to be used only when origin has been assigned to a different endpoint and you want to modify it
git remote -v #To see what are the origins
git add --all #adding new changes
git commit -am 'New repository in Github, from the Leonhard cluster' #Documenting new changes
git push origin master #propagating new changes. user and password will not be requested if remote origin and SSH keys have been properly provided
----
git push ##is possible if the following line has been applied
git push --set-upstream origin master

if still not working, then the alternative would be using jupyter locally and upload to GitHub







## Self study on Snakemake

A workflow is defined in a 'snakefiles'

A rule definition specifies (i)a name; (ii) any number of input and output files and (iii) either a shell command or Prython code that creares the output from input.

Generally the workflow for snakemake would be .fastq--> .sai(intermediate) -->.bam

The rule fastq_to_sai --> map the given .fastq file 

In [ ]:
rule fastq_to_sai:
    input: ref = REF, reads ="{sample}.{group}.fastq"
    output: temp("{sample}.{group}.sai")
    shell: "bwa aln {input.ref} {input.reads}> {output}"

BWA is used to read mapping, that produce suuffix array interval (.sai) and need to be converted to common format .bam

This is conducted by rule sai_to_bam

In [ ]:
rule sai_to_bam:
    input: REF, "{sample}.1.sai", "{sample}.2.sai", 
    "{sample}.1.fastq", "{sample}.2.fastq" 
    output: protected("{sample}.bam")
    shell: "bwa sampe {input} j samtools view -Sbh -4{output}"


Install snakemake in cluster using the following command (important: condo cannot install snakemake directly and we need to use mamba as a suppliment to install)

In [ ]:
$ conda install -n base -c conda-forge mamba
$ conda activate base # by default it should already be activated
$ mamba create -c conda-forge -c bioconda -n snakemake snakemake
$ conda activate snakemake
$ snakemake --help

change the repository of snakemake to a ideal one called snakemake_lsf in the cluster

In [ ]:
cd ~/.config/
mkdir snakemake
cd snakemake/
git clone https://github.com/AnimalGenomicsETH/snakemake_lsf.git # for some reason, only https works for me not the git@github.com
cd snakemake_lsf/


## Working with bcftools in the cluster

To learn a bit more about bcftools, several steps in conducted in the cluster of some .vcf.gz files
We first working in the headmode (may not be allowed in some cases)

1. First step is to create the index file for all vcf files

In [ ]:
bcftools index [file name.vcf.gz] ## This create vcf.gz.csi by default; tbi can be created if use -tbi/-t

2. Then we retrieve the stats file for each of the vcf file

In [ ]:
bcftools stats [file name.vcf.gz] > [file name.stats] 
                                ## This gives the statistic results and store the file as .stats in the directory

3. Combine multiple vcf files into one file (usually compressed)

In [ ]:
bcftools concat -Oz -o [the new file name.vcf.gz] [file name1.vcf.gz] [file name2.vcf.gz] 
                                 ## Concat combine multiple files togther as one compressed file

4. Selection of columns in the combined vcf files

In [ ]:
bcftools view -s RMXXXX [the combined file.vcf.gz] -Oz -o RMXXXX.vcf.gz
## This select the specific RMXXXX sample from the concat file and store as compressed file

###Important: remember to create the index for the combined file first otherwise it won't work

5. Selection of chromosomes in the file for one sample

In [ ]:
bcftools view -r 5 [BSWXXXXXX.vcf.gz] -Oz -o chromosome_5.vcf.gz

## This select specific colomn of the chromosome number on the file and saved as compressed.

6. Selection of columns with specific quality in the file

In [ ]:
bcftools view -i 'REF="C"'[chromosome_5.vcf.gz] -Oz -o REF_C.vcf.gz

## This select the columns with reference sequence to be C in the chromosome_5 file.

###This could also be conducted through awk


After all these steps, we used index for the newly created files and move on to the comp. mode, the first thing is to submit the job.

we first created a new directory with exact same *vcf.gz files and all comp.mode jobs are completed in that directory. 




In [ ]:
bsub "bcftools index filename.vcf.gz"
## this submit jobs to another device and run it there

bjobs ## Check if there is still unfinished jobs and show the status of unfinished jobs

bpeek #display the output of the job

Each of the job generating a report as lsf.o22545* file when the job is finished

In [ ]:
less(lsf.022545XXXXX) ## To see specific report
grep 'Successfully completed.' lsf.022545* # Show all successfully completed works
rm lsf.022545* #After checking it, the report can be removed.

Similar commends as the headmode is used to send jobs from step 1-6.




Snakemake is introduced in this case as it could run jobs individually once the output is ready and therefore it could also be used in this case.

Firstly we need to settle up snakemake in the cluster

In [ ]:
conda activate snakemake ## same as the installation process

Then we need to create a directory inside the VCF folder for snakemake (snakie)
After set up the directory in snakie, we can see several files created: snake_submit and Snakefile

However, one problem is that once a different environment is created, the bcftools is not installed and therefore we need to run the installation again for later use

One advantage is to allow different version of package used in seperate environment

In [ ]:
conda install -c bioconda bcftools

Inside the Snakefile, three rules already created to conduct the first 3 steps similar to using bcftools in the headmode as well as comp.mode

Generally, the structure of the Snakefile

In [ ]:
vcfs_origin

vcfs_end

rule all ## the general rule that applies to all rules created
##all the final output we need with all rules

rule [rule name]
    input: 
        vcf = # the file used
    output:
        vcf = # the output file name,
        csi = # the output file index form
        stats = # the output file with stats form
    threads: N ## number of thread used to run the rule
    resources:
        mem_mb = 500,  ## the memory required
        walltime = '00:20' # the wall clock time 
    shell: ## the command to be run to retrieve data
        ## usually bcftools commands but sometimes needs to change the vcfs_origin or vcfs_end to run properly
        '''
        bcftools index {input.vcf} ## this vcf is connected to the naming in input \n \
        bcftools stats {input.vcf} > {output.stats}
        '''

If the output of a rule before is used as the input of the next rule, we could write:

In [ ]:
input:
    vcf = rules.[rule name].output.vcf

Here are some commands to use after we finish edit the Snakefile (sometimes vim is not a good choice to edit as it won't automatically set everything, alternatives such as sublime text is used locally to edit the rules)

In [ ]:
snakemake -n # can be used as a dry-run and see if the workflow is defined properly.
snakemake -cores N/-cN ## the number of CPU cores to be used at the same time.
snakemake -R [specific rule] ## this allow you to run specific rules in snakemake

In order to run successfully, we need to edit rule all by first creating the workflow chart (DAG) and put all the last-step output in the rule all part.


In our case the flow for all six steps can be distinguished and the output we need would be the stats file for "REF_C.stats" and "RM1900.stats"; since "BSWXXXXXX.vcf.gz" is used for rules followed, we also need to put "BSWXX.stats" in the rule all to get the data we want.









Another useful tool to introduce would be the Linux Screen as a similar function of running jobs distantly.

In [ ]:
screen ## this create a new screen with default name and we can run all snakemake command in it when we are away.

In [ ]:
screen -S [file name] ## this create a new named session with the name you want

In [ ]:
[ctrl] a+d ## detach the screen session and it will continue running when you are away

In [ ]:
screen -r ## resume the session; if multiple sessions are ongoing, the ID needs to be specified after r 

In [ ]:
screen ls ## list all the current sessions


After we detach the session, the snakemake commands is continuing running and we could eventually get all the results in our VCF folder.
